In [1]:
# This file queries ski resort data from the ski-tracker-master-db (Mongo) and
# uses the lat/long coordinates to collect snow forecast data. Data for American
# ski resorts was collected from the National Weather Service API. Information
# about the API can be found here:
#
# https://www.weather.gov/documentation/services-web-api
#
# Weather for Canadian resorts was collected from xml documents found here:
#
# https://dd.weather.gc.ca/citypage_weather/xml/
#
# Information can about Canadian weather data can be found at:
#
# https://api.weather.gc.ca/
#
# The snow forecast data is then added to the ski resort data pulled from the Mongo
# database. The file then creates a JSON file containing all the data, which can then be read
# by the javascript file to create the map.

# Import required dependencies
from pymongo import MongoClient
from pprint import pprint
from bson import json_util
import requests
import json
import xmltodict

In [2]:
# Connect to Mongo database and pprint the resorts
mongo = MongoClient(port=27017)
db = mongo['ski-tracker-master-db']
resorts = list(db.resort_master.find())
pprint(resorts)

[{'_id': ObjectId('65aed7c8f3d05baae89cbbdc'),
  'country': 'US',
  'location': {'latitude': 48.302353, 'longitude': -117.564011},
  'name': '49 Degrees North',
  'pass_type': None,
  'region': 'WA',
  'slug': '49-degrees-north',
  'url': 'https://api.skiapi.com/v1/resort/49-degrees-north'},
 {'_id': ObjectId('65aed7c8f3d05baae89cbbdd'),
  'country': 'US',
  'location': {'latitude': 39.148201, 'longitude': -120.150902},
  'name': 'Alpine Meadows',
  'pass_type': None,
  'region': 'CA',
  'slug': 'alpine-meadows',
  'url': 'https://api.skiapi.com/v1/resort/alpine-meadows'},
 {'_id': ObjectId('65aed7c8f3d05baae89cbbde'),
  'country': 'US',
  'location': {'latitude': 40.583828, 'longitude': -111.63726},
  'name': 'Alta',
  'pass_type': 'Ikon',
  'region': 'UT',
  'slug': 'alta',
  'url': 'https://api.skiapi.com/v1/resort/alta'},
 {'_id': ObjectId('65aed7c8f3d05baae89cbbdf'),
  'country': 'US',
  'location': {'latitude': 60.961445, 'longitude': -149.102676},
  'name': 'Alyeska',
  'pass_ty

In [3]:
# Define a function that checks whether either of two words are contained in a string. This
# will be used to determine whether snow/flurries are reported in the weather forecast.
def is_word_in_string(word1, word2, input_string):
    return ((word1 in input_string) or (word2 in input_string))

# Use lat/long coordinates to query the weather forecast API for each resort
for resort in resorts:
    lat = round(resort['location']['latitude'], 2)
    long = round(resort['location']['longitude'], 2)

    # First handle the Canadian resorts using the Canadian governemtn xml files
    if resort['country'] == 'CA':

        # Given the name of the resort, assign an appropriate Canadian weather url corresponding to an
        # xml page closest to the resort's lat/long coordinates        
        if resort['slug'] == 'big-white':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000772_e.xml"
        elif resort['slug'] == 'bluemountain':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/ON/s0000108_e.xml"
        elif resort['slug'] == 'canada-olympic-park':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/AB/s0000126_e.xml"
        elif resort['slug'] == 'cypress-mountain':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000865_e.xml"
        elif resort['slug'] == 'grouse':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000141_e.xml"
        elif resort['slug'] == 'manning-park':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000496_e.xml"
        elif resort['slug'] == 'mt-seymour':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000373_e.xml"
        elif resort['slug'] == 'owlshead':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/QC/s0000345_e.xml"
        elif resort['slug'] == 'red-mountain':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000471_e.xml"
        elif resort['slug'] == 'sasquatch':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000226_e.xml"
        elif resort['slug'] == 'silver-star':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000324_e.xml"
        elif resort['slug'] == 'sun-peaks':
           canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000324_e.xml"
        elif resort['slug'] == 'sunshine-village':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/AB/s0000404_e.xml"
        elif resort['slug'] == 'tremblant':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/QC/s0000213_e.xml"
        elif resort['slug'] == 'whistler-blackcomb':
            canadian_url = f"https://dd.weather.gc.ca/citypage_weather/xml/BC/s0000078_e.xml"

        # Use the Canadian url to request weather data 
        try:
            canadian_data = requests.get(canadian_url).content
            canadian_dict = xmltodict.parse(canadian_data)

            resort['snow_firstperiod'] = is_word_in_string('snow', 'flurries', canadian_dict['siteData']['forecastGroup']['forecast'][1]['textSummary'])
            resort['snow_secondperiod'] = is_word_in_string('snow', 'flurries', canadian_dict['siteData']['forecastGroup']['forecast'][2]['textSummary'])
            resort['snow_thirdperiod'] = is_word_in_string('snow', 'flurries', canadian_dict['siteData']['forecastGroup']['forecast'][3]['textSummary'])
            resort['snow_fourthperiod'] = is_word_in_string('snow', 'flurries', canadian_dict['siteData']['forecastGroup']['forecast'][4]['textSummary'])

            resort['forecast_firstperiod'] = canadian_dict['siteData']['forecastGroup']['forecast'][1]['textSummary']
            resort['forecast_secondperiod'] = canadian_dict['siteData']['forecastGroup']['forecast'][2]['textSummary']
            resort['forecast_thirdperiod'] = canadian_dict['siteData']['forecastGroup']['forecast'][3]['textSummary']
            resort['forecast_fourthperiod'] = canadian_dict['siteData']['forecastGroup']['forecast'][4]['textSummary']

        # Print an error message with the name of each resort that's missing weather forecast data
        except (requests.RequestException, KeyError, IndexError) as e:
            print(f"Error processing api request for {resort['name']}")

    # Next handle American ski resorts using the National Weather Service API
    if resort['country'] == 'US':

        # Query the National Weather Service api to get snowfall data at the lat/long coordinates
        weather_service_url = f"https://api.weather.gov/points/{lat},{long}"
    
        # Query the api for weather data
        try:
            weather_service_data = requests.get(weather_service_url).json()
            forecast_url = weather_service_data['properties']['forecast']
            forecast_data = requests.get(forecast_url).json()

            resort['precip_chance_firstperiod'] = forecast_data['properties']['periods'][0]['probabilityOfPrecipitation']['value']
            resort['precip_chance_secondperiod'] = forecast_data['properties']['periods'][1]['probabilityOfPrecipitation']['value']
            resort['precip_chance_thirdperiod'] = forecast_data['properties']['periods'][2]['probabilityOfPrecipitation']['value']
            resort['precip_chance_fourthperiod'] = forecast_data['properties']['periods'][3]['probabilityOfPrecipitation']['value']

            resort['forecast_firstperiod'] = forecast_data['properties']['periods'][0]['shortForecast']
            resort['forecast_secondperiod'] = forecast_data['properties']['periods'][1]['shortForecast']
            resort['forecast_thirdperiod'] = forecast_data['properties']['periods'][2]['shortForecast']
            resort['forecast_fourthperiod'] = forecast_data['properties']['periods'][3]['shortForecast']

            resort['snow_firstperiod'] = is_word_in_string('Snow', 'Flurries', forecast_data['properties']['periods'][0]['shortForecast'])
            resort['snow_secondperiod'] = is_word_in_string('Snow', 'Flurries', forecast_data['properties']['periods'][1]['shortForecast'])
            resort['snow_thirdperiod'] = is_word_in_string('Snow', 'Flurries', forecast_data['properties']['periods'][2]['shortForecast'])
            resort['snow_fourthperiod'] = is_word_in_string('Snow', 'Flurries', forecast_data['properties']['periods'][3]['shortForecast'])

        # Print an error message with the name of each resort that's missing weather forecast data
        except (requests.RequestException, KeyError, IndexError) as e:
            print(f"Error processing api request for {resort['name']}")

Error processing api request for Alyeska
Error processing api request for Brighton
Error processing api request for Cataloochee
Error processing api request for Cranmore
Error processing api request for Diamond Peak
Error processing api request for Gore Mountain
Error processing api request for Gunstock
Error processing api request for Hunter Mountain
Error processing api request for Killington
Error processing api request for Loveland
Error processing api request for Mad River Glen
Error processing api request for Mt Spokane
Error processing api request for Ragged Mountain
Error processing api request for Red Lodge Mountain
Error processing api request for Schweitzer
Error processing api request for Shawnee Mountain
Error processing api request for Snow Valley
Error processing api request for Sugarloaf
Error processing api request for Sunday River
Error processing api request for Vail
Error processing api request for Wildcat
Error processing api request for Windham Mountain


In [4]:
# Save all the data to a json file that can be read by resort_plot_logic.js
with open('resort_data.json', 'w') as json_file:
    json.dump(resorts, json_file, default=json_util.default)